### Boiler Plate

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import BernoulliRBM, MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics, linear_model

In [17]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential
from keras.utils import to_categorical

In [2]:
bin_train = np.genfromtxt('data/bindigit_trn.csv',delimiter=',',)
target_train = np.genfromtxt('data/targetdigit_trn.csv',delimiter=',',)
bin_test = np.genfromtxt('data/bindigit_tst.csv',delimiter=',',)
target_test = np.genfromtxt('data/targetdigit_tst.csv',delimiter=',',)

### DBN

In [ ]:
def report_DBM(rbms = [150,100,50], iterations = 50, batch_size = 64, learning_rate = 0.01, verbose = True):
    #Initialize pipe
    networks = []
    count = 1
    for hidden_dim in rbms:
        rbm = BernoulliRBM(n_components = hidden_dim, batch_size = batch_size, learning_rate = learning_rate,
                      n_iter = iterations, verbose = verbose, random_state = 0)
        networks.append(('rbm{}'.format(count),rbm))
        count += 1

    networks.append(('mlp',MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam',alpha=0.01)))
    model = Pipeline(networks)
    
    #Train model
    model.fit(bin_train, target_train)

    #Report results
    print(metrics.classification_report(
        target_test,
        model.predict(bin_test)))

In [ ]:
report_DBM(rbms = [], iterations = 100, verbose = False)

### Stacked Autoencoders

In [25]:
def report_stacked_autoencoders(autoencoders, iterations = 50, batch_size=64 , verbose = True, target_train=target_train):
    input_layer = Input(shape=(784,))
    layer = input_layer
    #encode
    for hidden_dim in autoencoders:
        layer = Dense(hidden_dim, activation = 'relu')(layer)
    autoencoders.pop(-1)
    #decode
    for hidden_dim in reversed(autoencoders):
        layer = Dense(hidden_dim, activation = 'relu')(layer)
    layer = Dense(784, activation = 'relu')(layer)
    out = Dense(10, activation='sigmoid')(layer)    

    target_train = to_categorical(target_train, num_classes=10)
    model = Model(input_layer, out)
    model.compile(optimizer = 'adadelta', loss = 'mean_squared_error')
    model.fit(bin_train,target_train,
                   epochs=iterations,
                   batch_size=batch_size,
                   shuffle=True,
                   verbose=verbose)

    predictions = np.argmax(model.predict(bin_test),axis=1)
    print(metrics.classification_report(target_test,predictions))

In [ ]:
report_stacked_autoencoders(autoencoders = [150,100,50], iterations =50)

### Deeper MLP

In [23]:
def report_MLP(layers, iterations = 50, batch_size = 64, verbose = True, target_train = target_train):
    model = Sequential()
    for layer in layers:
        model.add(layer)

    target_train = to_categorical(target_train, num_classes=10)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    model.fit(bin_train,target_train,epochs=iterations,
              batch_size=batch_size,verbose=verbose, shuffle=False)
    predictions = np.argmax(model.predict(bin_test),axis=1)
    print(metrics.classification_report(target_test,predictions))

In [24]:
layers = [
    Dense(150,input_shape=(784,), activation='relu'),
    Dropout(0.5),
    Dense(100,input_shape=(784,), activation='relu'),
    Dropout(0.5),
    Dense(50,input_shape=(784,), activation='relu'),
    Dense(10, activation='sigmoid'),
    ]

report_MLP(layers)

Epoch 1/50
8000/8000 [==============================] - 1s 151us/step - loss: 0.0968 - acc: 0.2347
Epoch 2/50
8000/8000 [==============================] - 0s 61us/step - loss: 0.0539 - acc: 0.6257
Epoch 3/50
8000/8000 [==============================] - 0s 62us/step - loss: 0.0331 - acc: 0.7956
Epoch 4/50
8000/8000 [==============================] - 0s 62us/step - loss: 0.0245 - acc: 0.8534
Epoch 5/50
8000/8000 [==============================] - 1s 63us/step - loss: 0.0195 - acc: 0.8828
Epoch 6/50
8000/8000 [==============================] - 0s 62us/step - loss: 0.0169 - acc: 0.8988
Epoch 7/50
8000/8000 [==============================] - 0s 62us/step - loss: 0.0152 - acc: 0.9103
Epoch 8/50
8000/8000 [==============================] - 1s 63us/step - loss: 0.0140 - acc: 0.9161
Epoch 9/50
8000/8000 [==============================] - 1s 63us/step - loss: 0.0125 - acc: 0.9253
Epoch 10/50
8000/8000 [==============================] - 1s 65us/step - loss: 0.0117 - acc: 0.9313
Epoch 11/50
8000/8